In [1]:
import pandas as pd
import os

from copy import copy
from openpyxl import load_workbook
from openpyxl.styles import Border,Side,Alignment,Font,PatternFill
from openpyxl.utils import get_column_letter

In [2]:
month,year = 2,2024

prof_tax = 150

wages = pd.DataFrame({"CATEGORY":['SKILLED',"SEMI-SKILLED","UNSKILLED"],"Wage per day":[709,589,504]})

vendor_name = "M/S Uma Engineers,Sunabeda"

month_st_dt = pd.Period(f"{year}-{month}").start_time.date().strftime("%d-%m-%Y")
month_end_dt = pd.Period(f"{year}-{month}").end_time.date().strftime("%d-%m-%Y")

month_year = pd.Period(f"{year}-{month}").start_time.date().strftime("%b-%Y")
month_line = f"Due Wage Month:{month_year} from {month_st_dt} to {month_end_dt}"

vendor_name_line = f"Name of Contractor : {vendor_name}"

attendance_path = os.path.join("attendance",f"attendance_format_{month}_{year}.xlsx")

template_path  = "attendance.xlsx"

In [3]:
f"Name of Contractor : {vendor_name}"

'Name of Contractor : M/S Uma Engineers,Sunabeda'

In [4]:
def group_processing(group,category):
    group.drop(["SL NO"],axis=1,inplace=True)
    group["gross_wage"] = group["TOTAL PAY DAYS"] * group["Wage per day"] 
    group["nh_wage"] = group["NH DAY"] * group["Wage per day"] 
    group["gross_wage_pf"] = group["gross_wage"].apply(lambda x:15000 if x > 15000 else x)
    group["emp_epf"] = (group["gross_wage_pf"] * 0.12).round(2)
    group["emp_esi"] = (group["gross_wage"] * 0.0075).round(2)
    group["prof_tax"] =  group["gross_wage_pf"].apply(lambda x: prof_tax if x == 15000 else 0)
    group["emp_epf_esi_total"] = group["emp_epf"] + group["emp_esi"]
    group["empl_epf"] = (group["gross_wage_pf"] * 0.125).round(2)
    group["empl_edli"] = (group["gross_wage_pf"] * 0.005).round(2)
    group["empl_epf_edli_total"] = group["empl_epf"] + group["empl_edli"]
    group["empl_esi"] = (group["gross_wage"] * 0.0325).round(2)
    group["empl_epf_edli_esi_total"] = group["empl_epf"] + group["empl_edli"] + group["empl_esi"]
    group["net_pay"] = (group["gross_wage"] + group["nh_wage"]) - group["emp_epf_esi_total"] - group["prof_tax"]

    last_row = group.sum(axis=0).values
    last_row[:2] = ["",f"{category} TOTAL"]
    group.loc["total"] = last_row
    return group


In [5]:
def pf_esi_preprocessing(attendance_path,wages:pd.DataFrame):
    usecols=["SL NO","CATEGORY OF SKILLNESS","NAME OF CONTRACT PERSONNEL","TOTAL PAY DAYS","NH DAY"]
    data = pd.read_excel(attendance_path,header=7,usecols=usecols,skipfooter=7)
    
    merged_data = data.merge(wages,how="left",left_on="CATEGORY OF SKILLNESS",right_on="CATEGORY").drop(["CATEGORY"],axis=1)
    groups = merged_data.groupby("CATEGORY OF SKILLNESS")
    
    processed_df = pd.DataFrame()
    
    bill_pay_days ={}

    for category in merged_data["CATEGORY OF SKILLNESS"].unique():

        group = groups.get_group(category)
        processed_group = group_processing(group,category)
        bill_pay_days[category] = processed_group.loc["total","TOTAL PAY DAYS"]
        processed_df = pd.concat([processed_df,processed_group],axis=0)


    final_row = processed_df.loc["total"].sum(axis=0).values
    final_row[:2] = ["","TOTAL"]
    processed_df.loc["final_total"] = final_row
    processed_df.loc[["final_total","total"],"Wage per day"] = ""

    processed_df["SL NO"] = [int(val)+1 if val not in ["total","final_total"] else "" for val in processed_df.index]
    processed_df = pd.concat([processed_df.iloc[:,-1:],processed_df.iloc[:,:-1]],axis=1)

    empl_epf,empl_edli,esi_total = processed_df.loc["final_total",["empl_epf","empl_edli","empl_esi"]].values
    bill_pay_days["EMPL_PF"] = empl_epf
    bill_pay_days["EMPL_EDLI"] = empl_edli
    bill_pay_days["ESI"] = esi_total
    for category in ["SKILLED","SEMI-SKILLED","UNSKILLED"]:
        if category not in bill_pay_days.keys():
            bill_pay_days.update({category:0})

    processed_df.reset_index(drop=True,inplace=True)
    processed_df.drop(["CATEGORY OF SKILLNESS"],axis=1,inplace=True)

    rows_yellow_fill = processed_df[processed_df["NAME OF CONTRACT PERSONNEL"].str.contains("TOTAL")].index.values
    
    return processed_df,bill_pay_days,rows_yellow_fill

In [6]:
processed_df,bill_pay_days,rows_yellow_fill = pf_esi_preprocessing(attendance_path,wages)

C:\Users\CHINNUMINNU\AppData\Local\Temp\ipykernel_11708\2200540149.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group.drop(["SL NO"],axis=1,inplace=True)
C:\Users\CHINNUMINNU\AppData\Local\Temp\ipykernel_11708\2200540149.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group["gross_wage"] = group["TOTAL PAY DAYS"] * group["Wage per day"]
C:\Users\CHINNUMINNU\AppData\Local\Temp\ipykernel_11708\2200540149.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [7]:
processed_df

,SL NO,NAME OF CONTRACT PERSONNEL,TOTAL PAY DAYS,NH DAY,Wage per day,gross_wage,nh_wage,gross_wage_pf,emp_epf,emp_esi,prof_tax,emp_epf_esi_total,empl_epf,empl_edli,empl_epf_edli_total,empl_esi,empl_epf_edli_esi_total,net_pay
0,1,GURU TURUK,26,0,589,15314,0,15000,1800.0,114.85,150,1914.85,1875.00,75.00,1950.00,497.71,2447.71,13249.15
1,2,AJAY BENYA,25,0,589,14725,0,14725,1767.0,110.44,0,1877.44,1840.62,73.62,1914.24,478.56,2392.80,12847.56
2,3,PATI HIKKA,25,0,589,14725,0,14725,1767.0,110.44,0,1877.44,1840.62,73.62,1914.24,478.56,2392.80,12847.56
3,4,ANANTA HIAL,25,0,589,14725,0,14725,1767.0,110.44,0,1877.44,1840.62,73.62,1914.24,478.56,2392.80,12847.56
4,5,ANIL KUMAR SETHY,25,0,589,14725,0,14725,1767.0,110.44,0,1877.44,1840.62,73.62,1914.24,478.56,2392.80,12847.56
5,6,SURYAKANTA MOHANTY,25,0,589,14725,0,14725,1767.0,110.44,0,1877.44,1840.62,73.62,1914.24,478.56,2392.80,12847.56
6,7,GOVIND CH KRISHANI,25,0,589,14725,0,14725,1767.0,110.44,0,1877.44,1840.62,73.62,1914.24,478.56,2392.80,12847.56
7,8,CHANDRAKANTA NAYAK,25,0,589,14725,0,14725,1767.0,110.44,0,1877.44,1840.62,73.62,1914.24,478.56,2392.80,12847.56
8,9,UPENDRA PATRI,25,0,589,14725,0,14725,1767.0,110.44,0,1877.44,1840.62,73.62,1914.24,478.56,2392.80,12847.56
9,10,BIRAJ MUKHERJEE,25,0,589,14725,0,14725,1767.0,110.44,0,1877.44,1840.62,73.62,1914.24,478.56,2392.80,12847.56


In [20]:


bill_pay_days

{'SEMI-SKILLED': 251,
 'UNSKILLED': 125,
 'EMPL_PF': 26315.579999999998,
 'EMPL_EDLI': 1052.58,
 'ESI': 6852.250000000001,
 'SKILLED': 0}

In [15]:
def generate_pf_esi_sheet(template_path):
    '''
    This function reads attendance file and return styled excel sheet with values.
    '''
    workbook = load_workbook(template_path)
    max_name_length = max([len(name) for name in processed_df["NAME OF CONTRACT PERSONNEL"].values])

    sheet = workbook['PF_ESI']  # Update with your sheet's name
    sheet.column_dimensions["B"].width = (max_name_length+2)*1.2

    border_style = Border(left=Side(border_style="thin",color="FF000000"),
                              right=Side(border_style="thin",color="FF000000"),
                              top=Side(border_style="thin",color="FF000000"),
                              bottom=Side(border_style="thin",color="FF000000"))

    alignment_style = Alignment(horizontal='center',vertical="center")
    alignment_style_left = Alignment(horizontal='left',vertical="center")
    bold_font = Font(bold=True)
    yellow_fill = PatternFill(start_color="FFFF00",end_color="FFFF00",fill_type="solid")
    green_fill = PatternFill(start_color="00FF00",end_color="00FF00",fill_type="solid")

    cell = sheet.cell(row=2,column=1)
    cell.value = month_line

    cell = sheet.cell(row=3,column=1)
    cell.value = vendor_name_line

    start_row = 6
    start_column = 1  # Assuming you start from column A

    sheet.delete_rows(start_row,sheet.max_row)

    for index, row in processed_df.iterrows(): 
        for col_num, value in enumerate(row, start=start_column):   
            
            cell = sheet.cell(row=start_row + index, column=col_num)
            cell.value = value
            cell.font = bold_font

            if col_num != 1:
                cell.number_format = "0.00"

            if start_row + index - 6  in rows_yellow_fill:
                cell.fill = yellow_fill

            if col_num == 2:
                cell.alignment = alignment_style_left
                
            else:
                cell.alignment = alignment_style

    last_row = sheet[sheet.max_row]
    for cell in last_row:
        cell.fill = green_fill

    for row in sheet.iter_rows(min_row=start_row,min_col=1,max_row=5+len(processed_df),max_col=18):
            for cell in row:
                cell.border = border_style
                
    sheet.cell(row=16,column=2).value = "SEMI-SKILLED TOTAL"

    return workbook



In [16]:
wb = generate_pf_esi_sheet(template_path)

In [18]:
wb.save("pf_esi.xlsx")

In [256]:
def create_pf_esi_sheet(attendance_path,sheet):
    attendance_workbook = load_workbook(attendance_path)

    try:
        if attendance_workbook["PF_ESI"]:
            attendance_workbook.remove(attendance_workbook["PF_ESI"])
            target_sheet = attendance_workbook.create_sheet(title = "PF_ESI")
    except:
        target_sheet = attendance_workbook.create_sheet(title = "PF_ESI")

    for row in sheet.iter_rows(values_only=True):
        target_sheet.append(row)

    for row in range(1,sheet.max_row+1):
        for col in range(1,sheet.max_column+1):
            source_cell = sheet.cell(row=row,column=col)
            target_cell = target_sheet.cell(row=row,column=col)
            target_cell.number_format = source_cell.number_format
            target_cell.font = copy(source_cell.font)
            target_cell.alignment = copy(source_cell.alignment)
            target_cell.border = copy(source_cell.border)
            target_cell.fill = copy(source_cell.fill)

    for col_idx,column in enumerate(sheet.columns,start=1):
        target_sheet.column_dimensions[get_column_letter(col_idx)].width = sheet.column_dimensions[get_column_letter(col_idx)].width

    for merged_cell_range in sheet.merged_cells.ranges:
        target_sheet.merge_cells(merged_cell_range.coord)
        for merged_cell in merged_cell_range.cells:
            target_cell = target_sheet.cell(row=merged_cell[0],column=merge_cell[1])
            if merged_cell[0]  in [2,3]:
                target_cell.alignment = Alignment(horizontal="left",vertical="center")
            elif merged_cell[0]  in [4,5]:
                target_cell.alignment = Alignment(horizontal="center",vertical="center",wrap_text=True)               
            else:
                target_cell.alignment = Alignment(horizontal="center",vertical="center")

    attendance_workbook.save(attendance_path)
    return attendance_path

In [257]:
try:
    processed_df,bill_pay_days,rows_yellow_fill = pf_esi_preprocessing(attendance_path,wages)
    sheet = generate_pf_esi_sheet(template_path)
    target_path = create_pf_esi_sheet(attendance_path,sheet)
except Exception as e:
    print(e)

C:\Users\9216\AppData\Local\Temp\ipykernel_11140\2200540149.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group.drop(["SL NO"],axis=1,inplace=True)
C:\Users\9216\AppData\Local\Temp\ipykernel_11140\2200540149.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group["gross_wage"] = group["TOTAL PAY DAYS"] * group["Wage per day"]
C:\Users\9216\AppData\Local\Temp\ipykernel_11140\2200540149.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

C:\Users\9216\AppData\Local\Temp\ipykernel_11140\2200540149.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group["empl_epf_edli_esi_total"] = group["empl_epf"] + group["empl_edli"] + group["empl_esi"]
C:\Users\9216\AppData\Local\Temp\ipykernel_11140\2200540149.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group["net_pay"] = (group["gross_wage"] + group["nh_wage"]) - group["emp_epf_esi_total"] - group["prof_tax"]
C:\Users\9216\AppData\Local\Temp\ipykernel_11140\2200540149.py:19: SettingWithCopyW

In [243]:
processed_df

,SL NO,NAME OF CONTRACT PERSONNEL,TOTAL PAY DAYS,NH DAY,Wage per day,gross_wage,nh_wage,gross_wage_pf,emp_epf,emp_esi,prof_tax,emp_epf_esi_total,empl_epf,empl_edli,empl_epf_edli_total,empl_esi,empl_epf_edli_esi_total,net_pay
0,1,NITYA SUNDAR MUDULI,25.0,0,709,17725.0,0,15000.0,1800.00,132.94,150,1932.94,1875.00,75.00,1950.00,576.06,2526.06,15642.06
1,,SKILLED TOTAL,25.0,0,,17725.0,0,15000.0,1800.00,132.94,150,1932.94,1875.00,75.00,1950.00,576.06,2526.06,15642.06
2,2,JAGANNATH SAHU,25.0,0,589,14725.0,0,14725.0,1767.00,110.44,0,1877.44,1840.62,73.62,1914.24,478.56,2392.80,12847.56
3,,SEMI-SKILLED TOTAL,25.0,0,,14725.0,0,14725.0,1767.00,110.44,0,1877.44,1840.62,73.62,1914.24,478.56,2392.80,12847.56
4,3,DANA MAJHI,25.5,0,504,12852.0,0,12852.0,1542.24,96.39,0,1638.63,1606.50,64.26,1670.76,417.69,2088.45,11213.37
5,,UNSKILLED TOTAL,25.5,0,,12852.0,0,12852.0,1542.24,96.39,0,1638.63,1606.50,64.26,1670.76,417.69,2088.45,11213.37
6,,TOTAL,75.5,0,,45302.0,0,42577.0,5109.24,339.77,150,5449.01,5322.12,212.88,5535.00,1472.31,7007.31,39702.99


In [7]:
usecols=["SL NO","CATEGORY OF SKILLNESS","NAME OF CONTRACT PERSONNEL","TOTAL PAY DAYS","NH DAY"]
data = pd.read_excel(attendance_path,header=7,usecols=usecols,skipfooter=7)


wages = pd.DataFrame({"CATEGORY":['SKILLED',"SEMI-SKILLED","UNSKILLED"],"Wage per day":[709,589,504]})

merged_data = data.merge(wages,how="left",left_on="CATEGORY OF SKILLNESS",right_on="CATEGORY").drop(["CATEGORY"],axis=1)
groups = merged_data.groupby("CATEGORY OF SKILLNESS")

processed_df = pd.DataFrame()
bill_pay_days ={}

for category in merged_data["CATEGORY OF SKILLNESS"]:
    
    group = groups.get_group(category)
    processed_group = group_processing(group,category)
    bill_pay_days[category] = processed_group.loc["total","TOTAL PAY DAYS"]
    processed_df = pd.concat([processed_df,processed_group],axis=0)
    
    
final_row = processed_df.loc["total"].sum(axis=0).values
final_row[:2] = ["","TOTAL"]
processed_df.loc["final_total"] = final_row
processed_df.loc[["final_total","total"],"Wage per day"] = ""

processed_df["SL NO"] = [int(val)+1 if val not in ["total","final_total"] else "" for val in processed_df.index]
processed_df = pd.concat([processed_df.iloc[:,-1:],processed_df.iloc[:,:-1]],axis=1)

pf_edli_total,esi_total = processed_df.loc["final_total",["empl_epf_edli_total","empl_esi"]].values
bill_pay_days["EMPL_PF_EDLI_TOTAL"] = pf_edli_total
bill_pay_days["ESI"] = esi_total

processed_df.reset_index(drop=True,inplace=True)
processed_df.drop(["CATEGORY OF SKILLNESS"],axis=1,inplace=True)

rows_yellow_fill = processed_df[processed_df["NAME OF CONTRACT PERSONNEL"].str.contains("TOTAL")].index.values